In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [2]:
r = requests.get(constants.WILCO_CITY_XHR_URL, verify=False)
r.raise_for_status()

/Users/dan/code/texascovid19/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gis.wilco.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
city_data = r.json()['features']

In [4]:
city_data

[{'attributes': {'City_of_Residence': 'Austin', 'value': 5}},
 {'attributes': {'City_of_Residence': 'Cedar Park', 'value': 7}},
 {'attributes': {'City_of_Residence': 'Georgetown', 'value': 8}},
 {'attributes': {'City_of_Residence': 'Hutto', 'value': 1}},
 {'attributes': {'City_of_Residence': 'Leander', 'value': 6}},
 {'attributes': {'City_of_Residence': 'Round Rock', 'value': 7}}]

In [5]:
norm_city_data = []
for city in city_data:
        norm_city_data.append({
            'City of Residence': city['attributes']['City_of_Residence'],
            'Number of Cases': city['attributes']['value']
        })

In [8]:
df = pd.DataFrame(data=norm_city_data)

In [9]:
df

,City of Residence,Number of Cases
0,Austin,5
1,Cedar Park,7
2,Georgetown,8
3,Hutto,1
4,Leander,6
5,Round Rock,7


In [10]:
df.set_index("City of Residence", inplace=True)

In [11]:
today = date.today().isoformat()

In [12]:
df.to_csv(f"{constants.DATA_PATH}/wilco/bycity/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [14]:
df_timeseries = pd.read_csv(f"{constants.DATA_PATH}/wilco/bycity/timeseries.csv").set_index("City of Residence")

In [15]:
df.rename(columns={"Number of Cases": today}, inplace=True)

In [16]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [17]:
df_timeseries = pd.concat([df_timeseries, df], axis=1).rename_axis('City of Residence')

In [18]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [19]:
df_timeseries.to_csv(f"{constants.DATA_PATH}/wilco/bycity/timeseries.csv", header=True, index=True)